Human In The Loop

In [1]:
# The Problem
# Imagine you're building an agent that can send emails or make purchases. You don't want it to take these actions automatically - you want human approval first!

# Human-in-the-loop lets you:

# Pause execution for review
# Approve, reject, or edit actions
# Add safety controls to sensitive operations
# How It Works
# Agent encounters an interrupt() - execution pauses
# System surfaces information to human
# Human provides input (approve/reject/edit)
# Agent resumes with Command(resume=...)

In [2]:
from langchain_core.tools import tool
from langgraph.types import interrupt
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
llm = HuggingFaceEndpoint(
    repo_id='meta-llama/Llama-3.3-70B-Instruct',
    task="text-generation",
    max_new_tokens=2048,
)

model = ChatHuggingFace(llm = llm)

/home/rajgajjar04/Learnings/Langchain_Langgraph/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an email to a recipient."""

    approval = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "Do you want to send this email"
    })

    if approval.get("approved"):
        return f"Email sent to {to} with subject '{subject}'"
    else:
        return "Email cancelled by user"


In [6]:
from langchain.agents import create_agent
from langgraph.checkpoint.redis import RedisSaver

checkpointer = None
with RedisSaver.from_conn_string("redis://localhost:6379") as _checkpointer:
    _checkpointer.setup()
    checkpointer = _checkpointer

agent = create_agent(
    model=model,
    tools=[send_email],
    system_prompt = "You are a helpful email assistant. When asked to send emails, use the send_email tool.",
    checkpointer=checkpointer
)

In [7]:
from langchain.messages import HumanMessage

config = {
    'configurable': {
        'thread_id': 'email'
    }
}

result = agent.invoke({
    "messages": [HumanMessage(content="Send an email to alice@example.com with subject 'Meeting Tomorrow' and body 'Let's meet at 3pm.'")]
    },
    config=config
)

if "__interrupt__" in result:
    print("Agent paused for approval\n")

    interrupt_info = result["__interrupt__"][0]

    print("Interrupt details:")
    print(f"  To: {interrupt_info.value['to']}")
    print(f"  Subject: {interrupt_info.value['subject']}")
    print(f"  Body: {interrupt_info.value['body']}")
    print(f"  Message: {interrupt_info.value['message']}")
else:
    print("Agent completed without interrupt")

Agent paused for approval

Interrupt details:
  To: alice@example.com
  Subject: Meeting Tomorrow
  Body: Let's meet at 3pm.
  Message: Do you want to send this email


In [11]:
from langgraph.types import Command

result = agent.invoke(
    Command(resume={'approved': False}),
    config=config
)

print("Final response")
print(result['messages'][-1].content)

Final response

